## Preparación de Datos para Optimización de Cartera QUBO

El objetivo de esta fase es transformar los precios históricos brutos en los tres parámetros clave requeridos por nuestro modelo QUBO (Optimización Cuadrática Binaria Sin Restricciones).

Para ello se utiliza un enfoque de Backtesting, donde la optimización se realiza con datos de un período pasado (In-Sample) para validar su efectividad en un período futuro (Out-of-Sample).

Los tres inputs esenciales que obtendremos de la historia de precios son:



1.   Retorno Esperado ($r_i$): El beneficio esperado de cada activo.
2.   Coste ($c_i$): El precio de compra en el momento de la optimización.
3.   Matriz de Covarianza ($\sigma_{ij}$): El componente de riesgo que mide cómo se mueven los activos entre sí.

El siguiente código Python descarga y calcula estos parámetros utilizando un período de entrenamiento fijo (2023-2025) para simular una decisión de inversión tomada a principios de 2025.

In [ ]:
import yfinance as yf
import numpy as np

# --- 1. Definición de Parámetros de Entrada para Backtesting ---

# Tickers de activos (Sectores y Geografías)
# ^GSPC: S&P 500 (Índice USA) | RY: Royal Bank of Canada (Finanzas Canadá)
# NEE: NextEra Energy (Utilities/Energía USA) | DIS: Walt Disney Co (Entretenimiento)
# BABA: Alibaba (Tech China) | SNEJF: Sony Group Corp (Tech/Electro Japón)

TICKERS = ['^GSPC', 'RY', 'NEE', 'DIS', 'BABA', 'SNEJF']
START_DATE = '2023-01-01'
END_DATE = '2025-01-01'
INTERVALO_ANALISIS = '1d' # Diario


# --- 2. Descarga de Datos Históricos (Datasheet Crudo) ---

datos_crudos = yf.download(TICKERS,
                           start=START_DATE,
                           end=END_DATE,
                           interval=INTERVALO_ANALISIS,
                           auto_adjust=True)

precios = datos_crudos['Close'].dropna()


# --- 3. Derivación de Parámetros Clave para el Modelo (a 2025-01-01) ---

## PARÁMETRO 1: Coste (c_i)

# El coste c_i es el precio más reciente antes de la fecha de inversión (END_DATE).
costes_ci = precios.iloc[-1].rename('Coste ($c_i$)')


## PARÁMETRO 2: Retornos Diarios y Retorno Esperado (r_i)

# Calcular los retornos diarios logarítmicos
retornos = np.log(precios / precios.shift(1)).dropna()

# Multiplicamos por 252 (días de trading en un año) para anualizar.
roi = retornos.mean() * 252
roi.rename('Retorno Esperado ($r_i$ Anualizado)', inplace=True)


## PARÁMETRO 3: Matriz de Covarianza (Sigma_ij)

# La Matriz de Covarianza (Sigma) se calcula a partir de los retornos, anualizada
sigma_matrix = retornos.cov() * 252

print(f"\n- **Fecha de Optimización (Entrada del Modelo):** {END_DATE}")
print(f"\n- **r_i (Retorno Esperado):**\n{roi}")
print(f"\n- **c_i (Coste/Precio):**\n{costes_ci}")
print(f"\n- **Matriz Sigma (Covarianza):**\n{sigma_matrix}")

 ## Evaluación del Rendimiento de la Cartera (Backtesting Post-Inversión)

 Una vez que el modelo de optimización QUBO haya utilizado los datos históricos (hasta 2025-01-01) para seleccionar la cartera óptima de activos ($x_i$), el siguiente paso crucial es el Backtesting.

 Este código tiene como objetivo simular la inversión real midiendo el rendimiento acumulado que esos activos obtuvieron en el mercado en el tiempo posterior a la decisión.

 Al analizar el rendimiento en intervalos de 15 días, 1 mes, 2 meses, etc., podemos confirmar la efectividad de la optimización. Si la cartera seleccionada por el modelo supera consistentemente al mercado o a carteras aleatorias, se valida la calidad de la decisión.

 El código descarga los precios desde el día después de la inversión y calcula la tasa de crecimiento de cada activo en los puntos temporales definidos.

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

# Fecha de inicio del Backtesting (1  día después de la inversión)
START_BACKTEST = pd.to_datetime(END_DATE) + pd.Timedelta(days=1)

END_BACKTEST_MAX = pd.to_datetime('2025-07-02')

# Definición de los puntos de tiempo para el análisis
TIMEFRAMES = {
    '15 Días': START_BACKTEST + pd.Timedelta(days=15),
    'Mes 1': START_BACKTEST + relativedelta(months=1),
    'Mes 2': START_BACKTEST + relativedelta(months=2),
    'Mes 3': START_BACKTEST + relativedelta(months=3),
    'Mes 4': START_BACKTEST + relativedelta(months=4),
    'Mes 5': START_BACKTEST + relativedelta(months=5),
    'Mes 6': START_BACKTEST + relativedelta(months=6),
}


# 1. Descarga de Datos (Out-of-Sample)

datos_crudos_out = yf.download(
    TICKERS,
    start=START_BACKTEST.strftime('%Y-%m-%d'),
    end=END_BACKTEST_MAX.strftime('%Y-%m-%d'),
    interval=INTERVALO_ANALISIS,
    auto_adjust=True
)

precios_out = datos_crudos_out['Close'].dropna()

# Precio de Cierre el día de la inversión (P_inicial)
P_INICIAL = precios_out.iloc[0]


# 2. Cálculo del Rendimiento por Intervalo

resultados_rendimiento = {}

for label, end_date in TIMEFRAMES.items():
    # Encuentra la fila con la fecha más cercana a end_date
    P_FINAL_SERIE = precios_out.loc[precios_out.index <= end_date].iloc[-1]

    # Rendimiento Acumulado = ((P_final / P_inicial) - 1)
    rendimiento_pct = ((P_FINAL_SERIE / P_INICIAL) - 1)

    resultados_rendimiento[label] = rendimiento_pct.round(2)

# 3. Presentación de Resultados


print("\n Comportamiento del Mercado (Rendimiento Acumulado)")
print(f"Inversión Base (P_inicial): {START_BACKTEST.date()}")
print("-" * 60)


df_resultados = pd.DataFrame(resultados_rendimiento)
df_resultados = df_resultados.transpose()
df_resultados.index.name = 'Intervalo'

print(df_resultados)


In [ ]:
import pandas as pd
import numpy as np
from dimod import ExactSolver
from collections import defaultdict
import math

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN: Mapeo de Datos Reales al Modelo
# -----------------------------------------------------------------------------

# Usamos los datos obtenidos de yfinance en las celdas anteriores
assets = TICKERS  # Lista de nombres ['^GSPC', 'RY', ...]

# Convertimos las Series de Pandas a Diccionarios para acceso rápido
prices = costes_ci.to_dict()  # { 'BABA': 83.38, ... }
roi_dict = roi.to_dict()      # { 'BABA': -0.023, ... } (Renombramos para no confundir)
Sigma = sigma_matrix.values   # Convertimos el DataFrame de covarianza a matriz Numpy

# --- Parámetros de Inversión ---
# IMPORTANTE: Definimos un presupuesto C.
# Nota: Como usamos ExactSolver, mantén el presupuesto moderado para no
# generar demasiadas variables binarias (bits), o el solver tardará mucho.
C = 1000.0 

# Hiperparámetros del modelo
alpha = 0.5   # Penalización por desviarse del presupuesto (fuerza a gastar C)
beta  = 1.0   # Aversión al riesgo (peso del término de covarianza)

print(f"Configurando optimización para Presupuesto: ${C}")
print(f"Activos disponibles: {len(assets)}")

# -----------------------------------------------------------------------------
# 2. GENERACIÓN DE VARIABLES BINARIAS (Lotes)
# -----------------------------------------------------------------------------
# Replicamos la lógica de descomposición binaria con los datos reales

lots = []
for asset in assets:
    p = prices[asset]
    
    # Si el precio es mayor que el presupuesto, no podemos comprar ni una unidad
    if p > C:
        continue
        
    qmax = int(C // p)  # Máximo de acciones posibles
    
    if qmax <= 0:
        continue

    # Calcular cuántos bits (potencias de 2) necesitamos
    K = int(math.floor(math.log2(qmax))) 

    for k in range(K + 1):
        units = 2**k
        # Aseguramos no pasarnos del qmax total en la última potencia si fuera necesario, 
        # pero la descomposición estándar 2^k suele ser suficiente para aproximar.
        
        var_name = f"x_{asset}_{k}"
        cost = units * p
        
        # El retorno esperado total de este lote
        # Retorno unitario = ROI anualizado * Precio
        ret_expected = (roi_dict[asset] * p) * units
        
        lots.append({
            "var": var_name,
            "asset": asset,
            "k": k,
            "units": units,
            "price": p,
            "cost": cost,
            "ret": ret_expected
        })

print(f"Total de variables binarias generadas: {len(lots)}")
if len(lots) > 25:
    print("¡ADVERTENCIA! Más de 25 variables puede ser muy lento para ExactSolver.")

# -----------------------------------------------------------------------------
# 3. CONSTRUCCIÓN DE LA MATRIZ QUBO (Q)
# -----------------------------------------------------------------------------
Q = defaultdict(float)
asset_index = {a: i for i, a in enumerate(assets)}

# A) Término de Retorno (Maximizar retorno -> Minimizar negativo)
for row in lots:
    v = row["var"]
    Q[(v, v)] += -row["ret"]

# B) Término de Riesgo (Minimizar Varianza)
for u in lots:
    a_name = u["asset"]
    ia = asset_index[a_name]
    for v in lots:
        b_name = v["asset"]
        ib = asset_index[b_name]
        
        # Covarianza entre activo A y activo B
        cov_val = Sigma[ia, ib]
        
        # Coeficiente: beta * sigma * unidades_u * unidades_v
        coef = beta * cov_val * (u["units"] * v["units"])
        
        uvar, vvar = u["var"], v["var"]
        
        # Sumar al término correspondiente en la matriz triangular superior
        if uvar == vvar:
            Q[(uvar, uvar)] += coef
        elif uvar < vvar: # Solo una vez para pares distintos
            Q[(uvar, vvar)] += 2 * coef # x2 porque la matriz es simétrica
            
# C) Término de Presupuesto (Penalización alpha * (coste_total - C)^2)
# Expansión: alpha * [ sum(c_i^2 x_i) + 2*sum(c_i c_j x_i x_j) - 2*C*sum(c_i x_i) ]
for i, u in enumerate(lots):
    uvar = u["var"]
    cu = u["cost"]
    
    # Diagonal: alpha * cost^2 - 2 * alpha * C * cost
    Q[(uvar, uvar)] += alpha * (cu**2) - 2.0 * alpha * C * cu
    
    # Fuera de diagonal: 2 * alpha * cost_u * cost_v
    for j in range(i + 1, len(lots)):
        v = lots[j]
        vvar = v["var"]
        cv = v["cost"]
        
        # Key ordenada
        key = (uvar, vvar) if uvar <= vvar else (vvar, uvar)
        Q[key] += 2.0 * alpha * cu * cv

# -----------------------------------------------------------------------------
# 4. RESOLUCIÓN Y VISUALIZACIÓN DE RESULTADOS
# -----------------------------------------------------------------------------

# Resolver QUBO
print("Resolviendo QUBO (esto puede tardar unos segundos)...")
solver = ExactSolver()
sampleset = solver.sample_qubo(Q)
best = sampleset.first
solution = best.sample
energy = best.energy

print("Mejor energía encontrada:", energy)

# Filtrar variables activas
active_vars = [v for v, val in solution.items() if val == 1]

# Reconstruir cartera
units_by_asset = {a: 0 for a in assets}
cost_by_asset  = {a: 0.0 for a in assets}
ret_by_asset   = {a: 0.0 for a in assets}
lot_by_var = {row["var"]: row for row in lots}

for v in active_vars:
    info = lot_by_var[v]
    a = info["asset"]
    units_by_asset[a] += info["units"]
    cost_by_asset[a]  += info["cost"]
    ret_by_asset[a]   += info["ret"]

total_cost = sum(cost_by_asset.values())
total_ret  = sum(ret_by_asset.values())

# Calcular riesgo resultante
q_vec = np.array([units_by_asset[a] for a in assets], dtype=float)
final_risk = float(q_vec.T @ Sigma @ q_vec)

# Crear DataFrame de resumen
summary_df = pd.DataFrame({
    "Ticker": assets,
    "Unidades": [units_by_asset[a] for a in assets],
    "Precio Unit.": [prices[a] for a in assets],
    "Inversión ($)": [cost_by_asset[a] for a in assets],
    "Retorno Esp. ($)": [ret_by_asset[a] for a in assets],
})

# Formateo visual
print("\n=== CARTERA ÓPTIMA (QUBO) BASADA EN DATOS REALES ===")
display(summary_df[summary_df['Unidades'] > 0]) # Solo mostramos lo que compramos

print("-" * 40)
print(f"Presupuesto Objetivo: ${C}")
print(f"Inversión Total:      ${total_cost:.2f}")
print(f"Retorno Esperado:     ${total_ret:.2f}")
print(f"Riesgo (Varianza):    {final_risk:.6f}")

if abs(total_cost - C) < (C * 0.05):
    print("Estado: Presupuesto utilizado correctamente.")
else:
    print("Estado: Inversión baja. Intenta aumentar 'alpha' o revisar precios vs presupuesto.")

# Benchmark para código QUBO sin variables de holgura (con igualdad en la restricción del presupuesto).